In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))





DGL backend not selected or invalid.  Assuming PyTorch for now.
/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
def po_linear_model_square_expo(graph, alpha=1, beta=1, sigma=0.1, hete=0.5, gamma=2):    
    hete = 0.5
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * (sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i])**2   

In [ ]:
# unit-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

for seed in tqdm(range(repeat_num)):    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_nodes,))   
    
    for num_step in range(len(ramps)):
        print(f"Seed: {seed}")        
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            z = (rollout_index < np.quantile(rollout_index, p))            
            nx.set_node_attributes(g, 0, "z")            
            nx.set_node_attributes(g, {unit:1 for unit in range(num_nodes) if z[unit]}, "z")
            po_linear_model_square_expo(g, gamma = 1)            
            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
        
    
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/Square_incre_2step_CR_unit.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 8.1175
Epoch 0, Treat Prop: 0.02, Loss: 2.3869
Epoch 50, Treat Prop: 0.00, Loss: 0.1764
Epoch 50, Treat Prop: 0.02, Loss: 0.1791
Epoch 100, Treat Prop: 0.00, Loss: 0.0240
Epoch 100, Treat Prop: 0.02, Loss: 0.0351
Epoch 150, Treat Prop: 0.00, Loss: 0.0131
Epoch 150, Treat Prop: 0.02, Loss: 0.0195
Epoch 200, Treat Prop: 0.00, Loss: 0.0118
Epoch 200, Treat Prop: 0.02, Loss: 0.0158
Epoch 250, Treat Prop: 0.00, Loss: 0.0115
Epoch 250, Treat Prop: 0.02, Loss: 0.0139
Epoch 300, Treat Prop: 0.00, Loss: 0.0107
Epoch 300, Treat Prop: 0.02, Loss: 0.0123
Epoch 350, Treat Prop: 0.00, Loss: 0.0110
Epoch 350, Treat Prop: 0.02, Loss: 0.0121
0.01159515231847763
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1203
Epoch 0, Treat Prop: 0.05, Loss: 2.4064
Epoch 50, Treat Prop: 0.00, Loss: 0.1634
Epoch 50, Treat Prop: 0.05, Loss: 0.2073
Epoch 100, Treat Prop: 0.00, Loss: 0.0243
Epoch 100, Treat Prop: 0.05, Loss: 0.0539
Epoch 150, Treat Prop: 0.00, Loss: 0.0145
Epoch 150, Treat P

  0%|          | 1/1000 [00:25<6:58:57, 25.16s/it]

...

0.04099693521857262
Seed: 224
Epoch 0, Treat Prop: 0.00, Loss: 8.1033
Epoch 0, Treat Prop: 0.02, Loss: 2.3941
Epoch 50, Treat Prop: 0.00, Loss: 0.1724
Epoch 50, Treat Prop: 0.02, Loss: 0.1807
Epoch 100, Treat Prop: 0.00, Loss: 0.0219
Epoch 100, Treat Prop: 0.02, Loss: 0.0352
Epoch 150, Treat Prop: 0.00, Loss: 0.0132
Epoch 150, Treat Prop: 0.02, Loss: 0.0210
Epoch 200, Treat Prop: 0.00, Loss: 0.0122
Epoch 200, Treat Prop: 0.02, Loss: 0.0172
Epoch 250, Treat Prop: 0.00, Loss: 0.0111
Epoch 250, Treat Prop: 0.02, Loss: 0.0147
Epoch 300, Treat Prop: 0.00, Loss: 0.0105
Epoch 300, Treat Prop: 0.02, Loss: 0.0132
Epoch 350, Treat Prop: 0.00, Loss: 0.0125
Epoch 350, Treat Prop: 0.02, Loss: 0.0152
0.012794189155101776
Seed: 224
Epoch 0, Treat Prop: 0.00, Loss: 8.1153
Epoch 0, Treat Prop: 0.05, Loss: 2.4303
Epoch 50, Treat Prop: 0.00, Loss: 0.1654
Epoch 50, Treat Prop: 0.05, Loss: 0.2086
Epoch 100, Treat Prop: 0.00, Loss: 0.0245
Epoch 100, Treat Prop: 0.05, Loss: 0.0550
Epoch 150, Treat Prop: 0.00

 22%|██▎       | 225/1000 [1:31:42<4:59:45, 23.21s/it]

0.042172450572252274
Seed: 225
Epoch 0, Treat Prop: 0.00, Loss: 8.1208
Epoch 0, Treat Prop: 0.02, Loss: 2.3937
Epoch 50, Treat Prop: 0.00, Loss: 0.1653
Epoch 50, Treat Prop: 0.02, Loss: 0.1802
Epoch 100, Treat Prop: 0.00, Loss: 0.0231
Epoch 100, Treat Prop: 0.02, Loss: 0.0369
Epoch 150, Treat Prop: 0.00, Loss: 0.0134
Epoch 150, Treat Prop: 0.02, Loss: 0.0202
Epoch 200, Treat Prop: 0.00, Loss: 0.0122
Epoch 200, Treat Prop: 0.02, Loss: 0.0163
Epoch 250, Treat Prop: 0.00, Loss: 0.0119
Epoch 250, Treat Prop: 0.02, Loss: 0.0146
Epoch 300, Treat Prop: 0.00, Loss: 0.0107
Epoch 300, Treat Prop: 0.02, Loss: 0.0129
Epoch 350, Treat Prop: 0.00, Loss: 0.0116
Epoch 350, Treat Prop: 0.02, Loss: 0.0132
0.01599295251071453
Seed: 225
Epoch 0, Treat Prop: 0.00, Loss: 8.1299
Epoch 0, Treat Prop: 0.05, Loss: 2.3997
Epoch 50, Treat Prop: 0.00, Loss: 0.1581
Epoch 50, Treat Prop: 0.05, Loss: 0.2075
Epoch 100, Treat Prop: 0.00, Loss: 0.0240
Epoch 100, Treat Prop: 0.05, Loss: 0.0537
Epoch 150, Treat Prop: 0.00

 23%|██▎       | 226/1000 [1:32:06<5:02:52, 23.48s/it]

0.04138703644275665
Seed: 226
Epoch 0, Treat Prop: 0.00, Loss: 8.1245
Epoch 0, Treat Prop: 0.02, Loss: 2.3912
Epoch 50, Treat Prop: 0.00, Loss: 0.1712
Epoch 50, Treat Prop: 0.02, Loss: 0.1796
Epoch 100, Treat Prop: 0.00, Loss: 0.0239
Epoch 100, Treat Prop: 0.02, Loss: 0.0365
Epoch 150, Treat Prop: 0.00, Loss: 0.0133
Epoch 150, Treat Prop: 0.02, Loss: 0.0202
Epoch 200, Treat Prop: 0.00, Loss: 0.0122
Epoch 200, Treat Prop: 0.02, Loss: 0.0164
Epoch 250, Treat Prop: 0.00, Loss: 0.0117
Epoch 250, Treat Prop: 0.02, Loss: 0.0143
Epoch 300, Treat Prop: 0.00, Loss: 0.0129
Epoch 300, Treat Prop: 0.02, Loss: 0.0149
Epoch 350, Treat Prop: 0.00, Loss: 0.0112
Epoch 350, Treat Prop: 0.02, Loss: 0.0126
0.011835099197924137
Seed: 226
Epoch 0, Treat Prop: 0.00, Loss: 8.1119
Epoch 0, Treat Prop: 0.05, Loss: 2.4151
Epoch 50, Treat Prop: 0.00, Loss: 0.1581
Epoch 50, Treat Prop: 0.05, Loss: 0.2087
Epoch 100, Treat Prop: 0.00, Loss: 0.0239
Epoch 100, Treat Prop: 0.05, Loss: 0.0543
Epoch 150, Treat Prop: 0.00